# Distribución y correlaciones

## Indice
* [Fechas de publicación](#Fechas-de-publicación)
* [Géneros](#Géneros)
* [Número de páginas](#Número-de-páginas)
* [Formatos](#Formatos)
* [Libros mejor valorados](#Libros-mejor-valorados)

## Fechas de publicación

Se comienza importando las librerías necesarias para llevar a cabo el estudio:

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np

A continuación se carga el dataset:

In [ ]:
df = pd.read_csv("books.csv")
df.info()

Existen dos columnas relacionadas con las fechas de publicación, **publication_date** y **original_publication_date**. Para comprobar cuál es más interesante para el estudio, se analizan los nulos en cada una de ellas:

In [ ]:
print("Nulos en la columna *publication_date*: {}".format(df.publication_date.isnull().sum()))
print("Nulos en la columna *original_publication_date*: {}".format(df.original_publication_date.isnull().sum()))

Dado que **original_publication_date** tiene sensiblemente menos nulos, se escoge esta columna para el estudio. Se obtiene el dataframe que interesa, convirtiendo la columna **original_publication_date** a tipo *fecha* y eliminando los nulos:

In [ ]:
s_dates = pd.to_datetime(df.original_publication_date, errors="coerce")
s_dates.dropna(inplace=True)
print("Nulos en la columna *original_publication_date*: {}".format(s_dates.isnull().sum()))
s_dates.head()

Se comprueba cual es el valor máximo y mínimo de la serie:

In [ ]:
print("Fecha más reciente: {}".format(s_dates.max()))
print("Fecha más antigua: {}".format(s_dates.min()))

Se observa que existen libros cuya fecha original de publicación es el año 2068, por lo que se decide eliminar aquellos libros con fecha posterior al 2019. Para acotar el estudio al siglo XVIII y en adelante, también se decide eliminar aquellos libros con fecha de publicación anterior al año 1900:

In [ ]:
mask = ((s_dates > datetime.strptime("2020", '%Y')) | (s_dates < datetime.strptime("1900", '%Y')))
s_dates.drop(s_dates.loc[mask].index, inplace=True)

print("Los 5 más recientes:\n{}".format(s_dates.sort_values(ascending=False)[:5]))
print("Los 5 más antiguos:\n{}".format(s_dates.sort_values(ascending=True)[:5]))

Se procede a agrupar los resultados por año de publicación:

In [ ]:
s_dates = s_dates.groupby(s_dates.dt.year).agg('count')
s_dates.head()

A continuación se muestra esta información representada en un gráfico de barras:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors

norm = colors.Normalize(0, 1750)

def get_color(value):
         """Return color."""
         return plt.cm.viridis(norm(value))

bar_colors = [get_color(i) for i in s_dates.values]

fig = plt.figure(figsize=(25,10))
plt.bar(s_dates.index, s_dates.values, color=bar_colors)

# Configuramos la figura
plt.title('Publicaciones anuales')
plt.xlabel('Fecha')
plt.ylabel('Frecuencia (nº de libros)')
plt.xticks(ticks=s_dates.index, rotation='vertical')
plt.grid()
    
plt.show()

### Conclusiones

Tal y como se aprecia en la figura anterior, el año 2006 es el año con más publicaciones dentro de los datos obtenidos en el dataset.

## Géneros

A continuación se estudiará la distribución de los genéros dentro del dataset objeto de estudio.

Lo primero que se puede comprobar es que la variable *genres* es de tipo multivaluado:

In [ ]:
df.genres.head()

Por otra parte, también se observa que la columna cuenta con nulos:

In [ ]:
df.genres.isna().sum()

En este caso, para el estudio sobre los géneros, interesa sustituir los valores nulos por el literal *unknown*. Se llevará a cabo haciendo una copia del dataframe, únicamente de la columna de interés, y así mantener el original intacto:

In [ ]:
df_genres = df[["genres"]].copy()
df_genres.fillna(value="unknown", inplace=True)
print("Nulos en la columna genres: {}".format(df_genres.genres.isna().sum()))
df_genres.head(3)

Sobre el dataframe de estudio, se utiliza la función *get_dummies* que extraerá todos los géneros a columnas y se eliminará la columna original *genres*:

In [ ]:
df_genres = df_genres.join(df_genres.genres.str.get_dummies(sep=","))
df_genres.drop(columns=["genres"], inplace=True)
df_genres.columns.values

A continuación se obtiene una Serie con el sumatorio de cada una de las columnas; Es decir, el número de libros asociados a cada uno de los géneros. Se ordena en orden descendente:

In [ ]:
s_genres = df_genres.sum()
s_genres.sort_values(inplace=True, ascending=False)

s_genres

Con el objetivo de generar el gráfico de barras correspondiente a los géneros más populares históricamente, se reducirá la muestra a aquellos que pertenecen al cuarto cuartil, el 25% de elementos con mayor frecuencia:

In [ ]:
min_val = s_genres.quantile(.75)
min_val

In [ ]:
s_genres = s_genres.loc[s_genres >= min_val]
s_genres.values

A continuación el gráfico con la información de los géneros:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

norm = colors.Normalize(s_genres.min(), s_genres.max())

def get_color(value):
         """Return color."""
         return plt.cm.viridis(norm(value))

bar_colors = [get_color(i) for i in s_genres.values]

fig = plt.figure(figsize=(20,3))
plt.bar(s_genres.index, s_genres.values, color=bar_colors)

# Configuramos la figura
plt.title('Géneros más populares')
plt.xlabel('Géneros')
plt.ylabel('Frecuencia')
plt.xticks(rotation='vertical')
plt.grid()

plt.show()

Por último, se muestra esta misma información, en esta ocasión con una nube de palabras:

In [ ]:
from wordcloud import WordCloud

wordcloud = WordCloud(
    background_color="white",
    max_font_size=40,
    relative_scaling=.5
).generate(str(s_genres))

plt.figure(figsize=(10,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

### Conclusiones

Como se observa en ambas figuras, los 10 géneros más populares que aparecen en el dataset son:

In [ ]:
s_genres[:10]

## Número de páginas

En este punto se va a analizar la distribución del número de páginas de los libros.

Primero se observan los valores nulos del dataset para esta variable:

In [ ]:
df.num_pages.isnull().sum()

Se obtiene la serie sobre la que se va a realizar el estudio, se eliminan los nulos que no aportan valor al estudio y se cambia el tipo a int16:

In [ ]:
s_pages = df.num_pages.dropna()
print("Nulos en la serie: {}".format(s_pages.isnull().sum()))
s_pages = s_pages.astype(np.int16)
s_pages.head()

Información estadística sobre el número de páginas de los libros:

In [ ]:
s_pages.describe()

## Formatos

El objetivo de este punto es analizar el dataset en función del formato de los libros.

Para ello, obtenemos la serie de los formatos, habiendo sustituido los nulos por el valor *unknown*:

In [ ]:
s_format = df.format.fillna("unknown")
print("Nulos: {}".format(s_format.isnull().sum()))
s_format.unique()

A continuación se obtiene el número de libros por cada formato:

In [ ]:
s_format_count = s_format.value_counts()
s_format_count

Ya que aparecen muchos formatos que no nos aportan información al estudio, se obtienen aquellos correspondientes con el último cuartil más representativo:

In [ ]:
s_format_count = s_format_count.sort_values(ascending=False).loc[s_format_count.values > s_format_count.quantile(.75)]
s_format_count

A continuación se muestra el gráfico de barras con esta información:

In [ ]:
%matplotlib inline

norm = colors.Normalize(s_format_count.min(), s_format_count.max())

def get_color(value):
         """Return color."""
         return plt.cm.viridis(norm(value))

bar_colors = [get_color(i) for i in s_format_count.values]

fig = plt.figure(figsize=(15,5))
plt.bar(s_format_count.index, s_format_count.values, color=bar_colors)

# Configuramos la figura
plt.title('Formato de los libros')
plt.xlabel('Formato')
plt.ylabel('Frecuencia')
plt.xticks(ticks=s_format_count.index, rotation='vertical')
plt.grid()
    
plt.show()

Tal y como muestra la figura anterior, los 10 formatos más comunes son (sin tener encuenta los *unknowns*):
1. Paperback
2. Hardcover
3. Mass Market Paperback
4. Unknown Binding
5. Audio CD
6. Audiobook
7. Audio Cassette
8. Board Book
9. Audio
10. Spiral-bound

# Libros mejor valorados

En este punto se va a analizar los libros con mejor valoración del dataset.

Para ello, se obtiene la vista del dataset compuesta por las columnas:
* Título
* Valoración media
* Número de valoraciones
* Suma de valoraciones

Se ordenan los resultados en orden descendente y se muestran los primeros:

In [ ]:
df_ratings = df[["original_title", "authors", "average_rating", "ratings_count", "ratings_sum"]].sort_values(by=["ratings_sum"], ascending=False).head(100)
best_ratings = df_ratings.groupby(['original_title']).size()
best_ratings = best_ratings.sort_values(ascending=False)
best_ratings

A continuación se muestra un gráfico de barras con la misma información:

In [ ]:
from matplotlib import colors

norm = colors.Normalize(best_ratings.min(), best_ratings.max())

def get_color(value):
         """Return color."""
         return plt.cm.viridis(norm(value))

bar_colors = [get_color(i) for i in best_ratings.values]

fig = plt.figure(figsize=(20,10))
plt.bar(best_ratings.index, best_ratings.values, color=bar_colors)

# Configuramos la figura
plt.title('Libros mejor puntuados')
plt.xlabel('Libros')
plt.ylabel('Puntuación total')
plt.xticks(rotation='vertical')
plt.grid()

plt.show()

### Conclusiones

La lista con los libros mejor valorados es:

1. Pride and Prejudice                                     10
2. Harry Potter and the Philosopher's Stone                 7
3. Little Women                                             5
4. Harry Potter and the Order of the Phoenix                5
5. Harry Potter and the Prisoner of Azkaban                 5
6. Memoirs of a Geisha                                      5
7. The Fellowship of the Ring                               5
8. The Da Vinci Code                                        4
9. Harry Potter and the Half-Blood Prince                   4
10. Harry Potter and the Chamber of Secrets                 4
11. Of Mice and Men                                         3
12. O Alquimista                                            3
13. Nineteen Eighty-Four                                    3
14. Lord of the Flies                                       3
15. The Lovely Bones                                        3
16. Het Achterhuis                                          3
17. An Excellent conceited Tragedie of Romeo and Iuliet     3
18. The Giver                                               3
19. The Hobbit, or There and Back Again                     2
20. The Lion, the Witch and the Wardrobe                    2
21. Harry Potter and the Goblet of Fire                     2
22. Angels & Demons                                         2
23. Jane Eyre                                               2
24. Animal Farm: A Fairy Story                              1
25. A Game of Thrones                                       1
26. The Kite Runner                                         1
27. Harry Potter and the Deathly Hallows                    1
28. The Lightning Thief                                     1
29. The Catcher in the Rye                                  1
30. To Kill a Mockingbird                                   1